In [1]:
%pip install tensorflow==2.4.1 

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.25
    Uninstalling flatbuffers-24.3.25:
      Successfully uninstalled flatbuffers-24.3.25
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.9.0.1 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.


In [2]:
%pip install opencv-python mediapipe scikit-learn matplotlib

  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 24.3.25 which is incompatible.


In [3]:
%pip install flatbuffers 1.12.0


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement 1.12.0 (from versions: none)
ERROR: No matching distribution found for 1.12.0


In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [5]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [6]:
# Function to perform detection using MediaPipe Holistic
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    image.flags.writeable = False  # Make image read-only to improve performance
    results = model.process(image)  # Perform detection
    image.flags.writeable = True  # Make image writable again
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image back to BGR
    return image, results

# Function to draw landmarks on the image
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)  # Draw face landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw pose landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  # Draw left hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  # Draw right hand landmarks


In [7]:
# Function to draw styled landmarks on the image
def draw_styled_landmarks(image, results):
    # Draw face landmarks with specific styling
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                               mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                               mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))

    # Draw pose landmarks with specific styling
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))

    # Draw left hand landmarks with specific styling
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))

    # Draw right hand landmarks with specific styling
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/Users/khade/OneDrive/Documents/UWI-Documents/Year3/Sem2/COMP3901/Final Project/MP_Data') 

# Actions that we try to detect
#actions= np.array(['Are you hungry','Do you have any pets'])
#actions = np.array(['How are you','Where is the library','My favorite color is green','What is your favorite color',
 #                   "No, I don't have any pets",'Yes I could go for a snack'])

actions = np.array(['Are you hungry','Do you have any pets','How are you','Where is the library','My favorite color is green','What is your favorite color',
                    "No, I don't have any pets",'Yes I could go for a snack'])

# Thirty videos worth of data
no_sequences = 15

# collect 15 frames from each video
sequence_length = 20

# Folder start
start_folder = 1

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [16]:
model.load_weights('/Users/khade/OneDrive/Documents/UWI-Documents/Year3/Sem2/COMP3901/Final Project/weights/dense/1501.h5')

In [17]:
from scipy import stats

In [31]:

cap = cv2.VideoCapture(0)
# 1. New detection variables

sequence = []
sentence = []
predictions = []
threshold = 0.6
frame_count = 0
frame_interval = 3  # Adjust this value based on your preference

desired_aspect_ratio = 16 / 9
desired_width = 1280
desired_height = int(desired_width * (9/16))  # Maintain 16:9 aspect ratio


# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    # Create a named window with a specific size
    cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('OpenCV Feed', desired_width, desired_height)

    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Increment frame count
        frame_count += 1

        # 2. Prediction logic after processing a certain number of frames
        if frame_count % frame_interval == 0:
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-20:]

            if len(sequence) == 20:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                
                # Display predicted text in a box at the top of the screen
                #cv2.rectangle(image, (0, 0), (image.shape[1], 50), (0, 0, 0), -1)  # Draw black rectangle
                #cv2.putText(image, actions[np.argmax(res)], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                if np.unique(predictions[-10:])[0]==np.argmax(res):
                    if res[np.argmax(res)] > threshold:

                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

                # Print predicted text
                #predicted_text = ' '.join(sentence)
                #print("Translation: ", predicted_text)



                # End the loop after making a prediction
                break
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, actions[np.argmax(res)] , (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


What is your favorite color
